# How to deal with high cardinality categorical variables

You may want to do query analysis to create a filter on a categorical column. One of the difficulties here is that you usually need to specify the EXACT categorical value. The issue is you need to make sure the LLM generates that categorical value exactly. This can be done relatively easy with prompting when there are only a few values that are valid. When there are a high number of valid values then it becomes more difficult, as those values may not fit in the LLM context, or (if they do) there may be too many for the LLM to properly attend to.

In this notebook we take a look at how to approach this.

## Setup
#### Install dependencies

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/core @langchain/community zod chromadb @faker-js/faker
</Npm2Yarn>
```

#### Set environment variables

```
# Optional, use LangSmith for best-in-class observability
LANGSMITH_API_KEY=your-api-key
LANGCHAIN_TRACING_V2=true
```

#### Set up data

We will generate a bunch of fake names

In [45]:
import { faker } from "@faker-js/faker";

const names = Array.from({ length: 10000 }, () => faker.person.fullName());

Let's look at some of the names

In [46]:
names[0]

"Dale Kessler"

In [47]:
names[567]

"Mrs. Chelsea Bayer MD"

## Query Analysis

We can now set up a baseline query analysis

In [48]:
import { z } from "zod";

const searchSchema = z.object({
    query: z.string(),
    author: z.string(),
})

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [49]:
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnablePassthrough, RunnableSequence } from "@langchain/core/runnables";

const system = `Generate a relevant search query for a library system`;
const prompt = ChatPromptTemplate.fromMessages(
    [
      ["system", system],
      ["human", "{question}"],
    ]
)
const llmWithTools = llm.withStructuredOutput(searchSchema, {
  name: "Search"
})
const queryAnalyzer = RunnableSequence.from([
  {
    question: new RunnablePassthrough(),
  },
  prompt,
  llmWithTools
]);

We can see that if we spell the name exactly correctly, it knows how to handle it

In [50]:
await queryAnalyzer.invoke("what are books about aliens by Jesse Knight")

{ query: "books about aliens", author: "Jesse Knight" }

The issue is that the values you want to filter on may NOT be spelled exactly correctly

In [51]:
await queryAnalyzer.invoke("what are books about aliens by jess knight")

{ query: "books about aliens", author: "Jess Knight" }

### Add in all values

One way around this is to add ALL possible values to the prompt. That will generally guide the query in the right direction

In [52]:
const system = `Generate a relevant search query for a library system using the 'search' tool.

The 'author' you return to the user MUST be one of the following authors:

{authors}

Do NOT hallucinate author name!`
const basePrompt = ChatPromptTemplate.fromMessages(
    [
      ["system", system],
      ["human", "{question}"],
    ]
)
const prompt = await basePrompt.partial({ authors: names.join(", ") })

In [53]:
const queryAnalyzerAll = RunnableSequence.from([
  {
    question: new RunnablePassthrough(),
  },
  prompt,
  llmWithTools
])

However... if the list of categoricals is long enough, it may error!

In [54]:
try {
    const res = await queryAnalyzerAll.invoke("what are books about aliens by jess knight")
} catch (e) {
    console.error(e)
}

Error: 400 This model's maximum context length is 16385 tokens. However, your messages resulted in 49822 tokens (49792 in the messages, 30 in the functions). Please reduce the length of the messages or functions.
    at Function.generate (file:///Users/bracesproul/Library/Caches/deno/npm/registry.npmjs.org/openai/4.28.4/error.mjs:40:20)
    at OpenAI.makeStatusError (file:///Users/bracesproul/Library/Caches/deno/npm/registry.npmjs.org/openai/4.28.4/core.mjs:256:25)
    at OpenAI.makeRequest (file:///Users/bracesproul/Library/Caches/deno/npm/registry.npmjs.org/openai/4.28.4/core.mjs:299:30)
    at eventLoopTick (ext:core/01_core.js:63:7)
    at async file:///Users/bracesproul/Library/Caches/deno/npm/registry.npmjs.org/@langchain/openai/0.0.15/dist/chat_models.js:650:29
    at async RetryOperation._fn (file:///Users/bracesproul/Library/Caches/deno/npm/registry.npmjs.org/p-retry/4.6.2/index.js:50:12) {
  status: 400,
  headers: {
    "access-control-allow-origin": "*",
    "alt-svc": 'h3=

We can try to use a longer context window... but with so much information in there, it is not garunteed to pick it up reliably

```{=mdx}
<ChatModelTabs customVarName="llmLong" openaiParams={`{ model: "gpt-4-turbo-preview" }`} />
```

In [55]:
const structuredLlmLong = llmLong.withStructuredOutput(searchSchema, {
  name: "Search"
});
const queryAnalyzerAll = RunnableSequence.from([
  {
    question: new RunnablePassthrough(),
  },
  prompt,
  structuredLlmLong
]);

In [56]:
await queryAnalyzerAll.invoke("what are books about aliens by jess knight")

{ query: "aliens", author: "Jess Knight" }

### Find and all relevant values

Instead, what we can do is create an index over the relevant values and then query that for the N most relevant values,

In [57]:
import { Chroma } from "@langchain/community/vectorstores/chroma";
import { OpenAIEmbeddings } from "@langchain/openai";
import "chromadb";

const embeddings = new OpenAIEmbeddings({
  model: "text-embedding-3-small",
})
const vectorstore = await Chroma.fromTexts(names, {}, embeddings, {
  collectionName: "author_names"
})

[Module: null prototype] {
  AdminClient: [class AdminClient],
  ChromaClient: [class ChromaClient],
  CloudClient: [class CloudClient extends ChromaClient],
  CohereEmbeddingFunction: [class CohereEmbeddingFunction],
  Collection: [class Collection],
  DefaultEmbeddingFunction: [class _DefaultEmbeddingFunction],
  GoogleGenerativeAiEmbeddingFunction: [class _GoogleGenerativeAiEmbeddingFunction],
  HuggingFaceEmbeddingServerFunction: [class HuggingFaceEmbeddingServerFunction],
  IncludeEnum: {
    Documents: "documents",
    Embeddings: "embeddings",
    Metadatas: "metadatas",
    Distances: "distances"
  },
  JinaEmbeddingFunction: [class JinaEmbeddingFunction],
  OpenAIEmbeddingFunction: [class _OpenAIEmbeddingFunction],
  TransformersEmbeddingFunction: [class _TransformersEmbeddingFunction]
}

In [58]:
const selectNames = async (question: string) => {
    const _docs = await vectorstore.similaritySearch(question, 10);
    const _names = _docs.map(d => d.pageContent);
    return _names.join(", ");
}

In [59]:
const createPrompt = RunnableSequence.from([
    {
        question: new RunnablePassthrough(),
        authors: selectNames,
    },
    basePrompt
])

In [60]:
const queryAnalyzerSelect = createPrompt.pipe(llmWithTools);

In [61]:
await createPrompt.invoke("what are books by jess knight")

ChatPromptValue {
  lc_serializable: true,
  lc_kwargs: {
    messages: [
      SystemMessage {
        lc_serializable: true,
        lc_kwargs: {
          content: "Generate a relevant search query for a library system using the 'search' tool.\n" +
            "\n" +
            "The 'author' you ret"... 259 more characters,
          additional_kwargs: {}
        },
        lc_namespace: [ "langchain_core", "messages" ],
        content: "Generate a relevant search query for a library system using the 'search' tool.\n" +
          "\n" +
          "The 'author' you ret"... 259 more characters,
        name: undefined,
        additional_kwargs: {}
      },
      HumanMessage {
        lc_serializable: true,
        lc_kwargs: {
          content: "what are books by jess knight",
          additional_kwargs: {}
        },
        lc_namespace: [ "langchain_core", "messages" ],
        content: "what are books by jess knight",
        name: undefined,
        additional_kwargs: {}
  

In [62]:
await queryAnalyzerSelect.invoke("what are books about aliens by jess knight")

{ query: "books about aliens", author: "Jessica Kerluke" }